In [ ]:
import os
import time
import random
import pandas as pd
from tqdm.notebook import tqdm
from requests.exceptions import ReadTimeout, ConnectTimeout
import urllib3
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog, shotchartdetail, leaguedashplayerstats, playergamelogs

# ==========================================
# 1. 環境與參數設定
# ==========================================
SEASONS = ['2019-20', '2020-21', '2021-22', '2022-23', '2023-24']
TOP_N_PLAYERS = 150  # 每季抓場均得分前150名
DATASET_DIR = 'dataset'

# 建立資料夾
if not os.path.exists(DATASET_DIR):
    os.makedirs(DATASET_DIR)

GAMES_CSV_PATH = os.path.join(DATASET_DIR, 'games.csv')
SHOTS_CSV_PATH = os.path.join(DATASET_DIR, 'shots.csv')

# ==========================================
# 2. 輔助函式
# ==========================================

def fetch_with_retry(api_func, max_retries=3, **kwargs):
    """API請求包裝：含超時設定與隨機延遲重試"""
    kwargs['timeout'] = 25
    for i in range(max_retries):
        try:
            time.sleep(random.uniform(1.5, 3.0)) # 隨機延遲避免被封鎖
            return api_func(**kwargs)
        except (ReadTimeout, ConnectTimeout, urllib3.exceptions.ReadTimeoutError, ConnectionResetError):
            time.sleep(10) # 發生錯誤時休息久一點
        except Exception:
            break
    return None

def get_top_scorers(season, top_n=100):
    """取得該季得分榜前N名球員"""
    try:
        stats = leaguedashplayerstats.LeagueDashPlayerStats(season=season, per_mode_detailed='PerGame', timeout=30)
        df = stats.get_data_frames()[0]
        return df.sort_values(by='PTS', ascending=False).head(top_n)[['PLAYER_ID', 'PLAYER_NAME']].to_dict('records')
    except: return []

def clean_duplicates(filepath, subset_cols):
    """資料庫去重工具"""
    if os.path.exists(filepath):
        print(f"🧹 清理重複資料: {filepath} ...", end='\r')
        # 使用 low_memory=False 避免型別警告
        df = pd.read_csv(filepath, low_memory=False)
        df = df.drop_duplicates(subset=subset_cols, keep='last')
        df.to_csv(filepath, index=False)
        print(f"✅ 清理完成: {filepath}        ")

# ==========================================
# 3. 準備任務與檢查續傳
# ==========================================
print("步驟 1/3: 建立並過濾任務清單...")

# A. 建立所有目標任務 (球員 x 賽季)
target_player_ids = {} 
for season in SEASONS:
    for p in get_top_scorers(season, TOP_N_PLAYERS):
        target_player_ids[p['PLAYER_ID']] = p['PLAYER_NAME']

all_tasks = []
for pid, pname in target_player_ids.items():
    for season in SEASONS:
        all_tasks.append((str(pid), pname, str(season)))

# B. 讀取已完成進度 (防呆邏輯：強制轉字串 + 去除小數點)
processed_tasks = set()
if os.path.exists(GAMES_CSV_PATH):
    try:
        # 強制讀取為字串，避免 int/float 造成比對失敗
        df_exist = pd.read_csv(GAMES_CSV_PATH, usecols=['Player_ID', 'Season'], dtype=str, low_memory=False)
        if not df_exist.empty:
            for _, row in df_exist.iterrows():
                # 處理 '201939.0' 這種奇怪格式
                pid_clean = str(row['Player_ID']).split('.')[0].strip()
                season_clean = str(row['Season']).strip()
                processed_tasks.add((pid_clean, season_clean))
        print(f"🔄 已鎖定 {len(processed_tasks)} 個已完成任務 (將自動跳過)")
    except Exception as e:
        print(f"⚠️ 讀取舊檔異常 (將重新檢查): {e}")

# C. 過濾出剩餘任務
tasks_to_run = [t for t in all_tasks if (str(t[0]), str(t[2])) not in processed_tasks]
print(f"🚀 總任務: {len(all_tasks)} | 待執行: {len(tasks_to_run)}")

# ==========================================
# 4. 執行爬蟲 (手動更新進度條版)
# ==========================================

if not tasks_to_run:
    print("🎉 所有任務已完成！直接進入清理步驟。")
else:
    # dynamic_ncols=True 自動適應視窗寬度
    with tqdm(total=len(tasks_to_run), desc="初始化中", dynamic_ncols=True, unit="task") as pbar:
        
        for pid, pname, season in tasks_to_run:
            # 雙重檢查
            if (str(pid), str(season)) in processed_tasks:
                pbar.update(1); continue

            pbar.set_description(f"抓取: {pname} ({season})")
            
            batch_games = []
            batch_shots = []

            # --- A. 抓基礎數據 ---
            base_api = fetch_with_retry(playergamelog.PlayerGameLog, player_id=pid, season=season)
            if not base_api: 
                pbar.update(1); continue
            df_base = base_api.get_data_frames()[0]
            if df_base.empty: 
                pbar.update(1); continue

            # --- B. 抓進階數據並合併 ---
            adv_api = fetch_with_retry(
                playergamelogs.PlayerGameLogs, 
                player_id_nullable=pid, season_nullable=season,
                measure_type_player_game_logs_nullable='Advanced'
            )
            df_merged = df_base
            if adv_api:
                df_adv = adv_api.get_data_frames()[0]
                if not df_adv.empty:
                    df_base['Game_ID'] = df_base['Game_ID'].astype(str)
                    df_adv['GAME_ID'] = df_adv['GAME_ID'].astype(str)
                    
                    adv_cols = ['GAME_ID', 'OFF_RATING', 'DEF_RATING', 'NET_RATING', 'AST_PCT', 'AST_TO', 
                                'OREB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT', 'USG_PCT', 'PACE', 'PIE']
                    valid_cols = [c for c in adv_cols if c in df_adv.columns]
                    # 合併基礎與進階數據
                    df_merged = pd.merge(df_base, df_adv[valid_cols], left_on='Game_ID', right_on='GAME_ID', how='left')

            # --- C. 整理與產生 Label ---
            try: df_merged['GAME_DATE'] = pd.to_datetime(df_merged['GAME_DATE'])
            except: pass
            
            df_merged = df_merged.sort_values('GAME_DATE').reset_index(drop=True)
            df_merged['TARGET_PTS'] = df_merged['PTS'].shift(-1) # 下一場得分
            df_merged['Player_ID'] = pid
            df_merged['Player_Name'] = pname
            df_merged['Season'] = season
            df_merged = df_merged.dropna(subset=['TARGET_PTS']) # 移除無 Label 的最後一場
            batch_games.append(df_merged)

            # --- D. 抓投籃座標 ---
            shot_api = fetch_with_retry(
                shotchartdetail.ShotChartDetail,
                team_id=0, player_id=pid, 
                context_measure_simple='FGA', season_nullable=season
            )
            if shot_api:
                df_shots = shot_api.get_data_frames()[0]
                if not df_shots.empty:
                    s_cols = ['GAME_ID', 'LOC_X', 'LOC_Y', 'SHOT_MADE_FLAG', 'SHOT_TYPE', 'ACTION_TYPE']
                    valid_s = [c for c in s_cols if c in df_shots.columns]
                    batch_shots.append(df_shots[valid_s])

            # --- E. 增量存檔 (Checkpoint) ---
            if batch_games:
                df_g = pd.concat(batch_games, ignore_index=True)
                df_g.to_csv(GAMES_CSV_PATH, mode='a', header=not os.path.exists(GAMES_CSV_PATH), index=False)
            
            if batch_shots:
                df_s = pd.concat(batch_shots, ignore_index=True)
                df_s.to_csv(SHOTS_CSV_PATH, mode='a', header=not os.path.exists(SHOTS_CSV_PATH), index=False)

            # 更新記憶體狀態與進度條
            processed_tasks.add((str(pid), str(season)))
            pbar.update(1)

# ==========================================
# 5. 最終清理
# ==========================================
print("\n步驟 3/3: 資料庫去重與優化...")
# 確保同一場比賽只留一筆資料
clean_duplicates(GAMES_CSV_PATH, subset_cols=['Player_ID', 'Game_ID'])
clean_duplicates(SHOTS_CSV_PATH, subset_cols=None)

print("\n🎉 全部作業完成！資料庫已準備就緒。")